In [1]:
#the mdoel is utilsisng the pylcp package as the mmain package doing the calculations for the model the others are basic inbuilt packages
#the two data frames are decasy which a caculation of all the energy levels decay rates. 
# the others is all the possible decay rate from each levels and there corrosponidng decay rates
#functools packaga use cache 

import numpy as np
import matplotlib.pyplot as plt
import pylcp as pp
import scipy.constants as sp
import pandas as pd
import random
decays = pd.read_parquet('decays_ac.parquet')
df_ac = pd.read_excel('acditransitions.xlsx')
x = 100
#try for indinvidui l decays rates

In [2]:
#alpha is fine structre constant and rc is a unit conversion form au to hz
alpha = 1/137
Rc = ((4/3)*(alpha/219745)**3)*(1/(2.418e-17))

def g(i,j,med,jj):
    '''
    This allows for the cacultation of the decay rate in units of hz
    variables:
    i(flaot): this is the inintial energy levels ,units are cm^-1
    j(float):this is the level that it decays to in: cm^-2u
    med(float): dipole matrix elemnet in atomic units 
    jj(int): this it the total angular momentum
    '''
    a = ((i-j)**3)*(med)**2/(2*jj + 1)
    return a*Rc

In [3]:
def med(i,j):
    '''
    This returning the dipole matrix elemnt from a pandas data
    '''
    Eo = df_ac['E1'].tolist()
    if  i in Eo:
            c =  df_ac[(df_ac['E1'] == i) & (df_ac['E2'] == j)]['Med'].tolist()[0]# matrix dipole element for the pscific energy level
    else:
            c =  df_ac[(df_ac['E2'] == i) & (df_ac['E1'] == j)]['Med'].tolist()[0]

    return c


In [4]:
med(35850.31,11594.54)

5.8757

In [5]:
#the is the cell which defines the excited stated of the transition as the goal is to work all the levels that population can go to
#x is a fitler number saying we only what levels inlcuded that have population goint that is above 100 as the relates to less 0.000001% population 
x =  2000000
strp = decays[decays['E'] == 35850.31]['allowed transitions'].tolist()[0]
Eo = df_ac['E1'].tolist()
aalt = []
for i in strp:
        aalt.append(i)

In [6]:
#this loop caculates all the levels that it could go to fitlering for any levels whihc has a g_r < x,
# this is done by definening c which is obtained by loking through df_ac and finding the med 
x =  119700
strp = decays[decays['E'] == 35850.31]['allowed transitions'].tolist()[0]
Eo = df_ac['E1'].tolist()
aalt = []
grs = []
for i in strp:
        if i < 35850.31:
                aalt.append(i)
for p in range(10):
        for u in range(0 ,len(aalt)):
                if aalt[u] <= 35850.31:
                        kk = decays[decays['E'] == aalt[u]]['allowed transitions'].tolist()[0]
                        jj = decays[decays['E'] == aalt[u]]['j'].tolist()[0]
                        for quack in kk:
                                if quack < aalt[u]:
                                        c = med(aalt[u],quack)
                                        gr = g(aalt[u],quack,c,jj)# calcualtion of reduced energy levels
                                        if gr >= x:
                                                grs.append([aalt[u],quack,gr])#lis of all the enerfy levels
                                                aalt.append(quack)
                        aalt = list(set(aalt))#puts them in size order 
aalt.append(35850.31) # this is added speratly as it will look only through allowed transisitions     
aalt = sorted(list(set(aalt)))#gets rid of repeats
print(len(aalt))#a check if there lsit is correct lenght

7


In [7]:
len(aalt)

7

In [8]:
#get a list of all the other pepoerties needed such as total angular momnetum and the allowed transitions
jnm = []
gamman = []
allowed_transitions = []
fel = []
for i in aalt:
        fel.append(i)
        gamman.append(decays[decays['E'] == i]['gamma'].tolist()[0])
        jnm.append(decays[decays['E']==i]['j'].tolist()[0])
        allowed_transitions.append([i,decays[decays['E'] == i]['allowed transitions'].tolist()[0].tolist()])

In [9]:
#this gets rid of all levels that aren't un aalt
LOWERL = []
for i in  allowed_transitions:
    lower = []
    k = i[0]
    for j in i[1]:
        if j in aalt:
                lower.append(j)
    LOWERL.append(lower)
    allowed_transitions =  LOWERL

In [10]:
#cretaes a new data frame 
data = {'E': aalt ,'j':jnm,'gamma':gamman ,'allowed_transitions':allowed_transitions}
decaysf= pd.DataFrame(data)
print(decaysf)
decaysf.to_parquet('decaysf.parquet')

          E    j         gamma  \
0  11594.54  4.5  0.000000e+00   
1  16944.85  4.5  0.000000e+00   
2  25183.80  4.5  6.855080e+04   
3  27716.60  4.5  1.156258e+04   
4  29466.40  5.5  2.085434e+02   
5  32603.06  4.5  1.486347e+05   
6  35850.31  5.5  8.492392e+07   

                                 allowed_transitions  
0                                         [35850.31]  
1                                         [35850.31]  
2                                         [35850.31]  
3                                         [35850.31]  
4                                         [35850.31]  
5                                         [35850.31]  
6  [11594.54, 16944.85, 25183.8, 27716.6, 32603.0...  


Starting up the system  by setting up the hamiltonian

In [11]:
#actinium with the level 6D27S4F9/2 ->6d27p4G11/2 to start of with, make sure the decay is right at the moment of usse sodiums just get the code working. 
#all state constants are right except for the g factor need to change that they are still the ones for sodium
decays = pd.read_parquet('decays_ac.parquet')
decaysf = pd.read_parquet('decaysf.parquet')
print(decaysf)
det = 1
alpha = 1.0
hbar = sp.h/(2*np.pi)
gamma  = decaysf['gamma'].tolist()
J = decaysf['j'].tolist()
Ee = decaysf['E'].tolist()
I = 3/2
gj = 1.3126
gI  =  1.1/3.15245e-14*3/2
AHFS = (0.1712168e4)*10**6
#print(decays)

          E    j         gamma  \
0  11594.54  4.5  0.000000e+00   
1  16944.85  4.5  0.000000e+00   
2  25183.80  4.5  6.855080e+04   
3  27716.60  4.5  1.156258e+04   
4  29466.40  5.5  2.085434e+02   
5  32603.06  4.5  1.486347e+05   
6  35850.31  5.5  8.492392e+07   

                                 allowed_transitions  
0                                         [35850.31]  
1                                         [35850.31]  
2                                         [35850.31]  
3                                         [35850.31]  
4                                         [35850.31]  
5                                         [35850.31]  
6  [11594.54, 16944.85, 25183.8, 27716.6, 32603.0...  


In [12]:
#sets up the hyperfine manifolds for the hamilton
hamilton ={}
for i in range(len(gamma)):
    name = 'H_' + str(i)
    name2 = 'mu_q' + str(i)
    code = '{},{} = pp.hamiltonians.hyperfine_coupled(J[{}],I,gj,gI,AHFS)'.format(name,name2,i)
    exec(code)
    hamilton['H_' + str(i)] = globals()[name]

In [13]:
#set ups list of indices that corrospond to all the connection to the individual levels
allowed_transitions = decaysf['allowed_transitions'].tolist()
allow_i = []
kl = 0
for i in allowed_transitions:
      k = []
      if len(i) > 0:   
            for z in i:
                  k.append(Ee.index(z))
            allow_i.append([kl,k])
      kl = kl + 1      

In [14]:
allow_i

[[0, [6]],
 [1, [6]],
 [2, [6]],
 [3, [6]],
 [4, [6]],
 [5, [6]],
 [6, [0, 1, 2, 3, 5, 4]]]

In [15]:
#this then calculates all the dipole matrix elements for the different connections

dqij = {}
repeat = []
for qq in allow_i: 
    stat = qq[0]
    for kk in qq[1]:
        if stat > kk:
            #print(J[stat])
            name  = 'dq' + str(stat) + str(kk)
            name_o = 'dq' + str(kk) + str(stat)
            if name_o not in repeat:
                code  = '{} = pp.hamiltonians.dqij_two_hyperfine_manifolds(J[{}],J[{}],I)'.format(name,kk,stat)
                exec(code)
                dqij[name] = globals()[name]
                 
        else:
            #print(J[kk])
            name  = 'dq' + str(kk) + str(stat)
            name_o = 'dq' + str(stat) + str(kk)
            if name_o not in repeat:
                code  = '{} = pp.hamiltonians.dqij_two_hyperfine_manifolds(J[{}],J[{}],I)'.format(name,stat,kk)
                exec(code)
                dqij[name] = globals()[name]
                print(name)
        repeat.append(name)

dq60
dq61
dq62
dq63
dq64
dq65


In [16]:
#hyperfine energy levels 
EEE = {}
i = 0
for k in hamilton:
    name_v = 'E_'  + str(i)
    i = i + 1
    value_n = np.unique(np.diagonal(hamilton[k]))
    EEE[name_v] = value_n
print(EEE)

{'E_0': array([-1.4125386e+10, -7.2767140e+09,  1.2841260e+09,  1.1557134e+10]), 'E_1': array([-1.4125386e+10, -7.2767140e+09,  1.2841260e+09,  1.1557134e+10]), 'E_2': array([-1.4125386e+10, -7.2767140e+09,  1.2841260e+09,  1.1557134e+10]), 'E_3': array([-1.4125386e+10, -7.2767140e+09,  1.2841260e+09,  1.1557134e+10]), 'E_4': array([-1.6693638e+10, -8.1327980e+09,  2.1402100e+09,  1.4125386e+10]), 'E_5': array([-1.4125386e+10, -7.2767140e+09,  1.2841260e+09,  1.1557134e+10]), 'E_6': array([-1.6693638e+10, -8.1327980e+09,  2.1402100e+09,  1.4125386e+10])}


In [17]:
Ee[6]

35850.31

In [18]:
#magfiels which in this case is a 0 matrix as not including one 
magField = lambda R:np.zeros(R.shape)

In [19]:
#this adds all the hyperfine manifold caculated to a hamiltoniain called hamiltonians_ac_a
i = 0
hamiltonians_ac_a = pp.hamiltonian()
for M in range(len(gamma)):
    hamiltonians_ac_a.add_H_0_block('H_' + str(M), hamilton['H_' + str(M)])
    i = i + 1
    print('H_' + str(M))
print(i)

H_0
H_1
H_2
H_3
H_4
H_5
H_6
7


In [20]:
#this allows for the dipe matrix eelmtns ot be connected to each element in the matrix
z = 0
repeats = []
for i in allow_i:
    hope = i[0]
    e2 = 'H_' + str(hope)
    for kk in i[1]:
        e1 = 'H_' + str(kk)
        if hope > kk:
                ha = 'dq' + str(hope) + str(kk)
                ha_s = 'dq' + str(kk) + str(hope)# this is here to stop repeats being added
                repeats.append(ha)
                if ha_s in repeats:
                        cc = dqij[ha]
                        c = med(Ee[hope] ,Ee[kk])
                        gamma_r = g(Ee[hope],Ee[kk],c,J[hope])
                        hamiltonians_ac_a.add_d_q_block(e1,e2,cc, gamma = gamma_r)
                        print(gamma_r)
        else:
            #print(kk,hope)
            ha = 'dq' + str(kk) + str(hope)
            ha_s = 'dq' + str(hope) + str(kk)
            repeats.append(ha_s)
            if ha_s not in repeats:
                c = med(Ee[hope] ,Ee[kk])
                gamma_r = g(Ee[hope],Ee[kk],c,J[kk])
                cc = dqij[ha]
                hamiltonians_ac_a.add_d_q_block(e2,e1,cc, gamma = gamma_r)
                print(gamma_r)

82974933.13828921
690215.9098048583
873571.676822659
43476.15564248765
3447.126199149857
113437.02853685859


Now the laser is being set up so that it will excite the population for different levels 

In [21]:

#define the laser as a dictionairy the names in this have to corrospond to names defined in in the d_q add block if not specified default is ''g' 'e' labels 
e = (EEE['E_0'][3] - EEE['E_1'][3])
e2 =(EEE['E_1'][3] - EEE['E_0'][3])
def laser(delta,delta1,s1,s2,s3):
  laserbeam = {}
  laserbeam['H_0->H_6'] = pp.laserBeams(
        [{'kvec':np.array([1., 0., 0.]), 'pol':np.array([0, 1, 0.]),
          'pol_coord':'cartesian', 'delta':delta, 's':s1}],
        beam_type= pp.infinitePlaneWaveBeam)
  laserbeam['H_0->H_6'] = pp.laserBeams([{'kvec':np.array([0, 0, 1]), 'pol':np.array([0,0,1]),
       'pol_coord':'spherical', 'delta':delta1, 's':s3}],beam_type =  pp.infinitePlaneWaveBeam)
  #laserbeam['H_17->H_34'] = pp.laserBeams([{'kvec':np.array([0, 0, 1]), 'pol':np.array([0,0,1]),
       #'pol_coord':'spherical', 'delta':delta1, 's':s3}],beam_type =  pp.infinitePlaneWaveBeam)#s3 adds a re
  return laserbeam

In [22]:
#actinum scattering plot
# s = np.linspace(0.25,10,1000)
# g = []
# g1 =[]
# g2 = []
# g3 = []
# for i in range(len(s)):
#     print(i + 1)
#     laserbeam = laser(e,s[i],0,0)
#     rateeq = pp.rateeq(laserbeam , magField,Hamiltonian_ac)
#     N0  = np.zeros((rateeq.hamiltonian.n,))
#     N0[39] = 1
#     rateeq.set_initial_pop(N0)
#     rateeq.evolve_populations([0,50/gamma[0]],max_step=1/gamma[0])
#     #g.append(rateeq.sol.y[87][53])
#     #g1.append(rateeq.sol.y[72][53])
#     #
#     g2.append(rateeq.sol.y[71][53])
#    # g3.append(rateeq.sol.y[70][53])

In [23]:
# this caculates the all the mf levels labels as a list |n,F,m_f>
fff = []
M = 0
for i in aalt:
    H = decays[decays['E'] == i]['HF'].tolist()[0]
    for j in H:
        if j == 0:
            fff.append([M,j,0])  
        else:        
            for k in range(-j,j+1):
                 fff.append([M,j,k])
    M = M+1
print(len(fff))
maxi = len(fff) -1

296


In [24]:
# the laser is the established as and then inptued into a function which caclutlates the reate eqaution
laserbeam  = laser(e,e2,0,0,0)
rateeq = pp.rateeq(laserbeam,magField,hamiltonians_ac_a)

In [25]:
#this calculates the unit t_0  which is the time scale N0 is where all the intial population is put 
t_0 = 1/gamma[6]
N0  = np.zeros((rateeq.hamiltonian.n,))
N0[maxi] = 1
rateeq.set_initial_pop(N0)
rateeq.evolve_populations([0,0.000005/t_0],max_step=1)

KeyboardInterrupt: 

the initial population is now being put in and the popualtion them varrieed over time and plotted to see what happens

In [ ]:
# this plots the rate eqautions to visualize population evolution this will be filterd based on levels that don't get more thana certain population  
import matplotlib.colors as mcolors

used_colors = set()

def get_distinct_color():
    base_colors = list(mcolors.TABLEAU_COLORS.values())  # You can use other color palettes as well
    available_colors = [color for color in base_colors if color not in used_colors]

    if not available_colors:
        # Reset used_colors if all colors have been used
        used_colors.clear()
        available_colors = base_colors

    color = available_colors[0]
    used_colors.add(color)
    return color
 
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1)

i = 0
ee = 1
R = []
for bob in range(len(rateeq.sol.y)):
    ee = fff[bob][0]
    ff = fff[bob][1]
    m = fff[bob][2]
    k = rateeq.sol.y[bob][99]
    ll = rateeq.sol.y[bob][0]

    # Get a distinct color
 

    #Plot only if k > 0.01
    if k > 0.0001:
        color = get_distinct_color()
        ax.plot(1e9*t_0*rateeq.sol.t, rateeq.sol.y[bob,],'--',color = color, linewidth=1.0,
                label='|' + ','.join([str(ee), str(ff), str(m), str(bob)]) + '>')
        i += 3

plt.legend(fontsize='7.8', loc='upper right')
plt.title('Cooling Ac-227')
plt.xlabel('t($n$s)')
plt.ylabel('N')
plt.show()




In [ ]:
aalt

[11594.54, 16944.85, 25183.8, 27716.6, 29466.4, 32603.06, 35850.31]

In [ ]:
def b(t,m):
    v =((2*sp.k*t)/m)**(1/2)
    return v 



In [ ]:
v_s = b(1000,3.7694234e-25)
v_s

270.6569465989613

In [ ]:
N = 3.77e-25/(2*(2*np.pi*hbar))
def a(v,g,l):
  st = (v**2)*N*l/g
  return st 

In [ ]:
a(v_s,4e7,412e-9)

0.2146500064646314

In [ ]:
aa = []
for i in gg:
    aa.append(a(v_s,i,412e-9,))

NameError: name 'gg' is not defined

In [ ]:
gamma[6]
1/gamma[6]g
100*1/gamma[6]

1.1775246005690793e-06

In [ ]:
0.000005*gamma[6] 

424.6195788677008